In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
#import missingno as msno

# matplotlib
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

# scipy specifics
from scipy import stats as sps
from scipy.interpolate import interp1d

In [2]:
try:
    from urllib.request import Request, urlopen  # Python 3
except ImportError:
    from urllib2 import Request, urlopen  # Python 2

req = Request('https://cloud.minsa.gob.pe/s/AC2adyLkHCKjmfm/download')
req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
content = urlopen(req)

test = pd.read_csv(content , sep=";", na_values='EN INVESTIGACIÓN', encoding='utf-8')
print(test)

         FECHA_CORTE DEPARTAMENTO PROVINCIA                DISTRITO METODODX  \
0           20220210        JUNIN  HUANCAYO                EL TAMBO       AG   
1           20220210         LIMA      LIMA  SAN JUAN DE MIRAFLORES       AG   
2           20220210        PIURA     PIURA                CASTILLA       AG   
3           20220210       ANCASH     SANTA                CHIMBOTE       PR   
4           20220210         LIMA      LIMA                PUCUSANA       AG   
...              ...          ...       ...                     ...      ...   
3424888     20220210       CALLAO    CALLAO              BELLAVISTA      PCR   
3424889     20220210       CALLAO    CALLAO              BELLAVISTA      PCR   
3424890     20220210         LIMA      LIMA               LA MOLINA      PCR   
3424891     20220210        PIURA     PIURA   VEINTISEIS DE OCTUBRE       AG   
3424892     20220210         LIMA      LIMA                    LIMA      PCR   

         EDAD       SEXO  FECHA_RESULTA

In [3]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'].astype(str), format='%Y%m%d')
test.tail()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona
3424888,20220210,CALLAO,CALLAO,BELLAVISTA,PCR,42.0,MASCULINO,2021-04-08,70102.0,NaN
3424889,20220210,CALLAO,CALLAO,BELLAVISTA,PCR,82.0,FEMENINO,2021-04-22,70102.0,NaN
3424890,20220210,LIMA,LIMA,LA MOLINA,PCR,35.0,FEMENINO,2021-02-18,150114.0,NaN
3424891,20220210,PIURA,PIURA,VEINTISEIS DE OCTUBRE,AG,30.0,MASCULINO,NaT,200115.0,NaN
3424892,20220210,LIMA,LIMA,LIMA,PCR,36.0,MASCULINO,2021-12-19,150101.0,NaN


In [4]:
test.isnull().sum()

FECHA_CORTE             0
DEPARTAMENTO          312
PROVINCIA          163075
DISTRITO           163075
METODODX                0
EDAD                  347
SEXO                    1
FECHA_RESULTADO    993178
UBIGEO             163075
id_persona          64091
dtype: int64

In [5]:
indice_departamento = pd.read_csv('https://raw.githubusercontent.com/annaabsi/git-scraper-covid19/main/resultados/positivos_por_departamentos.csv')
indice_departamento

,DEPARTAMENTO,METODODX,POBLACION,INDICE
0,AMAZONAS,42543,426806,9968
1,ANCASH,120596,1180638,10214
2,APURIMAC,36699,430736,8520
3,AREQUIPA,203178,1497438,13568
4,AYACUCHO,47080,668213,7046
5,CAJAMARCA,93328,1453711,6420
6,CALLAO,143758,1129854,12724
7,CUSCO,113961,1357075,8398
8,HUANCAVELICA,24287,365317,6648
9,HUANUCO,50522,760267,6645


In [6]:
test = pd.merge(test, indice_departamento,
                        how="left", on=["DEPARTAMENTO"])
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
0,20220210,JUNIN,HUANCAYO,EL TAMBO,AG,68.0,FEMENINO,NaT,120114.0,24615058.0,122245.0,1361467.0,8979.0
1,20220210,LIMA,LIMA,SAN JUAN DE MIRAFLORES,AG,21.0,MASCULINO,NaT,150133.0,24731649.0,1545976.0,10628470.0,14546.0
2,20220210,PIURA,PIURA,CASTILLA,AG,20.0,MASCULINO,NaT,200104.0,24731665.0,152935.0,2047954.0,7468.0
3,20220210,ANCASH,SANTA,CHIMBOTE,PR,23.0,MASCULINO,2020-07-04,21801.0,24856166.0,120596.0,1180638.0,10214.0
4,20220210,LIMA,LIMA,PUCUSANA,AG,25.0,MASCULINO,NaT,150124.0,24856186.0,1545976.0,10628470.0,14546.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424888,20220210,CALLAO,CALLAO,BELLAVISTA,PCR,42.0,MASCULINO,2021-04-08,70102.0,NaN,143758.0,1129854.0,12724.0
3424889,20220210,CALLAO,CALLAO,BELLAVISTA,PCR,82.0,FEMENINO,2021-04-22,70102.0,NaN,143758.0,1129854.0,12724.0
3424890,20220210,LIMA,LIMA,LA MOLINA,PCR,35.0,FEMENINO,2021-02-18,150114.0,NaN,1545976.0,10628470.0,14546.0
3424891,20220210,PIURA,PIURA,VEINTISEIS DE OCTUBRE,AG,30.0,MASCULINO,NaT,200115.0,NaN,152935.0,2047954.0,7468.0


In [7]:
poblacion_csv = pd.read_csv('poblacion_provincia.csv')
poblacion_csv

,UBIGEO,PROVINCIA,POBLACION
0,10100,CHACHAPOYAS,63188
1,10200,BAGUA,84672
2,10300,BONGARA,26830
3,10400,CONDORCANQUI,51344
4,10500,LUYA,47827
...,...,...,...
191,240300,ZARUMILLA,56038
192,250100,CORONEL PORTILLO,447733
193,250200,ATALAYA,61049
194,250300,PADRE ABAD,77044


In [8]:
poblacion_dict = poblacion_csv.to_dict('split')
poblacion_dict['data']

[[10100, 'CHACHAPOYAS', 63188],
 [10200, 'BAGUA', 84672],
 [10300, 'BONGARA', 26830],
 [10400, 'CONDORCANQUI', 51344],
 [10500, 'LUYA', 47827],
 [10600, 'RODRIGUEZ DE MENDOZA', 33651],
 [10700, 'UTCUBAMBA', 119294],
 [20100, 'HUARAZ', 185276],
 [20200, 'AIJA', 6433],
 [20300, 'ANTONIO RAYMONDI', 13950],
 [20400, 'ASUNCION', 7710],
 [20500, 'BOLOGNESI', 24012],
 [20600, 'CARHUAZ', 50007],
 [20700, 'CARLOS FERMIN FITZCARRALD', 18496],
 [20800, 'CASMA', 57256],
 [20900, 'CORONGO', 8017],
 [21000, 'HUARI', 63264],
 [21100, 'HUARMEY', 33066],
 [21200, 'HUAYLAS', 56557],
 [21300, 'MARISCAL LUZURIAGA', 21787],
 [21400, 'OCROS', 7224],
 [21500, 'PALLASCA', 24371],
 [21600, 'POMABAMBA', 26675],
 [21700, 'RECUAY', 18085],
 [21800, 'SANTA', 474053],
 [21900, 'SIHUAS', 28630],
 [22000, 'YUNGAY', 55769],
 [30100, 'ABANCAY', 120116],
 [30200, 'ANDAHUAYLAS', 150758],
 [30300, 'ANTABAMBA', 11781],
 [30400, 'AYMARAES', 24570],
 [30500, 'COTABAMBAS', 55208],
 [30600, 'CHINCHEROS', 46544],
 [30700, 'GRAU

In [9]:
poblacion =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][2]
    poblacion.append(array)
    
poblacion

[63188,
 84672,
 26830,
 51344,
 47827,
 33651,
 119294,
 185276,
 6433,
 13950,
 7710,
 24012,
 50007,
 18496,
 57256,
 8017,
 63264,
 33066,
 56557,
 21787,
 7224,
 24371,
 26675,
 18085,
 474053,
 28630,
 55769,
 120116,
 150758,
 11781,
 24570,
 55208,
 46544,
 21759,
 1175765,
 61708,
 43690,
 34743,
 97458,
 16426,
 54851,
 12797,
 317801,
 32482,
 8341,
 97205,
 75277,
 51838,
 29139,
 9909,
 9292,
 19866,
 17063,
 388170,
 83167,
 83916,
 151714,
 29357,
 123948,
 83913,
 203724,
 145770,
 51678,
 47114,
 22638,
 38602,
 1129854,
 511019,
 24000,
 63131,
 71582,
 34754,
 106476,
 70143,
 62059,
 167910,
 26644,
 47579,
 101735,
 70043,
 121265,
 37503,
 53901,
 14588,
 33883,
 18182,
 85995,
 315799,
 53247,
 32427,
 16372,
 52095,
 138275,
 29160,
 50086,
 36987,
 17114,
 18705,
 445752,
 262110,
 78472,
 14832,
 174016,
 595183,
 59138,
 167385,
 88405,
 22757,
 239105,
 91849,
 40041,
 57604,
 1118724,
 123480,
 15982,
 86411,
 30987,
 85091,
 112970,
 85092,
 168670,
 55868

In [10]:
provincia =[]
for x in range(len(poblacion_dict['data'])):
    array = poblacion_dict['data'][x][1]
    provincia.append(array)
    
provincia

['CHACHAPOYAS',
 'BAGUA',
 'BONGARA',
 'CONDORCANQUI',
 'LUYA',
 'RODRIGUEZ DE MENDOZA',
 'UTCUBAMBA',
 'HUARAZ',
 'AIJA',
 'ANTONIO RAYMONDI',
 'ASUNCION',
 'BOLOGNESI',
 'CARHUAZ',
 'CARLOS FERMIN FITZCARRALD',
 'CASMA',
 'CORONGO',
 'HUARI',
 'HUARMEY',
 'HUAYLAS',
 'MARISCAL LUZURIAGA',
 'OCROS',
 'PALLASCA',
 'POMABAMBA',
 'RECUAY',
 'SANTA',
 'SIHUAS',
 'YUNGAY',
 'ABANCAY',
 'ANDAHUAYLAS',
 'ANTABAMBA',
 'AYMARAES',
 'COTABAMBAS',
 'CHINCHEROS',
 'GRAU',
 'AREQUIPA',
 'CAMANA',
 'CARAVELI',
 'CASTILLA',
 'CAYLLOMA',
 'CONDESUYOS',
 'ISLAY',
 'LA UNION',
 'HUAMANGA',
 'CANGALLO',
 'HUANCA SANCOS',
 'HUANTA',
 'LA MAR',
 'LUCANAS',
 'PARINACOCHAS',
 'PAUCAR DEL SARA SARA',
 'SUCRE',
 'VICTOR FAJARDO',
 'VILCAS HUAMAN',
 'CAJAMARCA',
 'CAJABAMBA',
 'CELENDIN',
 'CHOTA',
 'CONTUMAZA',
 'CUTERVO',
 'HUALGAYOC',
 'JAEN',
 'SAN IGNACIO',
 'SAN MARCOS',
 'SAN MIGUEL',
 'SAN PABLO',
 'SANTA CRUZ',
 'CALLAO',
 'CUSCO',
 'ACOMAYO',
 'ANTA',
 'CALCA',
 'CANAS',
 'CANCHIS',
 'CHUMBIVILCAS',


In [11]:
res = {provincia[i]: poblacion[i] for i in range(len(provincia))}
res

{'CHACHAPOYAS': 63188,
 'BAGUA': 84672,
 'BONGARA': 26830,
 'CONDORCANQUI': 51344,
 'LUYA': 47827,
 'RODRIGUEZ DE MENDOZA': 33651,
 'UTCUBAMBA': 119294,
 'HUARAZ': 185276,
 'AIJA': 6433,
 'ANTONIO RAYMONDI': 13950,
 'ASUNCION': 7710,
 'BOLOGNESI': 24012,
 'CARHUAZ': 50007,
 'CARLOS FERMIN FITZCARRALD': 18496,
 'CASMA': 57256,
 'CORONGO': 8017,
 'HUARI': 63264,
 'HUARMEY': 33066,
 'HUAYLAS': 56557,
 'MARISCAL LUZURIAGA': 21787,
 'OCROS': 7224,
 'PALLASCA': 24371,
 'POMABAMBA': 26675,
 'RECUAY': 18085,
 'SANTA': 474053,
 'SIHUAS': 28630,
 'YUNGAY': 55769,
 'ABANCAY': 120116,
 'ANDAHUAYLAS': 150758,
 'ANTABAMBA': 11781,
 'AYMARAES': 24570,
 'COTABAMBAS': 55208,
 'CHINCHEROS': 46544,
 'GRAU': 21759,
 'AREQUIPA': 1175765,
 'CAMANA': 61708,
 'CARAVELI': 43690,
 'CASTILLA': 34743,
 'CAYLLOMA': 97458,
 'CONDESUYOS': 16426,
 'ISLAY': 54851,
 'LA UNION': 12797,
 'HUAMANGA': 317801,
 'CANGALLO': 32482,
 'HUANCA SANCOS': 8341,
 'HUANTA': 97205,
 'LA MAR': 75277,
 'LUCANAS': 51838,
 'PARINACOCHAS':

In [12]:
test['POBLACION'] = test['PROVINCIA'].map(res)

In [13]:
null_columns = test.columns[test.isnull().any()]
test[test["SEXO"].isnull()][null_columns]

,DEPARTAMENTO,PROVINCIA,DISTRITO,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
2529107,LIMA,LIMA,LIMA,0.0,NaN,2021-03-13,150101.0,10881464.0,1545976.0,9674755.0,14546.0


In [14]:
#find_provincia = test['PROVINCIA'] == 'UCAYALI'
find_distrito = test['DEPARTAMENTO'] == 'LORETO'
find_fecha = test['FECHA_RESULTADO'] == '2022-01-02'

data_exploratoria = test[find_distrito & find_fecha]
data_exploratoria

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
213211,20220210,LORETO,MAYNAS,SAN JUAN BAUTISTA,PR,32.0,FEMENINO,2022-01-02,160113.0,20925980.0,57245.0,550551.0,5571.0
287780,20220210,LORETO,MAYNAS,PUNCHANA,PR,31.0,MASCULINO,2022-01-02,160108.0,22425039.0,57245.0,550551.0,5571.0


In [15]:
data_exploratoria.groupby("SEXO").count()

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
SEXO,,,,,,,,,,,,
FEMENINO,1,1,1,1,1,1,1,1,1,1,1,1
MASCULINO,1,1,1,1,1,1,1,1,1,1,1,1


In [16]:
test['SEXO'] = test['SEXO'].fillna('FEMENINO')

In [17]:
null_columns = test.columns[test.isnull().any()]
test[test["PROVINCIA"].isnull()][null_columns]

,DEPARTAMENTO,PROVINCIA,DISTRITO,EDAD,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE
20,LIMA,NaN,NaN,54.0,2020-11-21,NaN,24615020.0,1545976.0,NaN,14546.0
31,LIMA,NaN,NaN,39.0,2020-07-24,NaN,24833739.0,1545976.0,NaN,14546.0
38,LIMA,NaN,NaN,28.0,2022-01-21,NaN,24833787.0,1545976.0,NaN,14546.0
51,ICA,NaN,NaN,22.0,NaT,NaN,24662193.0,98796.0,NaN,10131.0
90,ICA,NaN,NaN,36.0,2020-09-12,NaN,24692780.0,98796.0,NaN,10131.0
...,...,...,...,...,...,...,...,...,...,...
3424606,LIMA,NaN,NaN,32.0,2022-01-24,NaN,NaN,1545976.0,NaN,14546.0
3424723,LIMA,NaN,NaN,30.0,2020-09-09,NaN,NaN,1545976.0,NaN,14546.0
3424760,LIMA,NaN,NaN,0.0,2022-02-02,NaN,NaN,1545976.0,NaN,14546.0
3424798,LIMA,NaN,NaN,0.0,2022-01-21,NaN,NaN,1545976.0,NaN,14546.0


In [18]:
test['FECHA_RESULTADO'] = pd.to_datetime(test['FECHA_RESULTADO'], errors='coerce', dayfirst=True)
test['FECHA_RESULTADO']

0                NaT
1                NaT
2                NaT
3         2020-07-04
4                NaT
             ...    
3424888   2021-04-08
3424889   2021-04-22
3424890   2021-02-18
3424891          NaT
3424892   2021-12-19
Name: FECHA_RESULTADO, Length: 3424893, dtype: datetime64[ns]

In [19]:
test['POBLACION_CIENMIL'] = test['POBLACION']/100000

#test = test.drop(labels="POBLACION_CIENMIL", axis=1)
test

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
0,20220210,JUNIN,HUANCAYO,EL TAMBO,AG,68.0,FEMENINO,NaT,120114.0,24615058.0,122245.0,595183.0,8979.0,5.95183
1,20220210,LIMA,LIMA,SAN JUAN DE MIRAFLORES,AG,21.0,MASCULINO,NaT,150133.0,24731649.0,1545976.0,9674755.0,14546.0,96.74755
2,20220210,PIURA,PIURA,CASTILLA,AG,20.0,MASCULINO,NaT,200104.0,24731665.0,152935.0,894847.0,7468.0,8.94847
3,20220210,ANCASH,SANTA,CHIMBOTE,PR,23.0,MASCULINO,2020-07-04,21801.0,24856166.0,120596.0,474053.0,10214.0,4.74053
4,20220210,LIMA,LIMA,PUCUSANA,AG,25.0,MASCULINO,NaT,150124.0,24856186.0,1545976.0,9674755.0,14546.0,96.74755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424888,20220210,CALLAO,CALLAO,BELLAVISTA,PCR,42.0,MASCULINO,2021-04-08,70102.0,NaN,143758.0,1129854.0,12724.0,11.29854
3424889,20220210,CALLAO,CALLAO,BELLAVISTA,PCR,82.0,FEMENINO,2021-04-22,70102.0,NaN,143758.0,1129854.0,12724.0,11.29854
3424890,20220210,LIMA,LIMA,LA MOLINA,PCR,35.0,FEMENINO,2021-02-18,150114.0,NaN,1545976.0,9674755.0,14546.0,96.74755
3424891,20220210,PIURA,PIURA,VEINTISEIS DE OCTUBRE,AG,30.0,MASCULINO,NaT,200115.0,NaN,152935.0,894847.0,7468.0,8.94847


In [20]:
salidasxsemanas = test.sort_values(by = 'FECHA_RESULTADO')
start_date = "2021-11-08"
end_date = "2022-02-06"

after_start_date = salidasxsemanas["FECHA_RESULTADO"] >= start_date
before_end_date = salidasxsemanas["FECHA_RESULTADO"] <= end_date
between_two_dates = after_start_date & before_end_date

filtered_dates = salidasxsemanas.loc[between_two_dates]

filtered_dates

,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
2449207,20220210,LA LIBERTAD,TRUJILLO,TRUJILLO,PCR,72.0,MASCULINO,2021-11-08,130101.0,10367661.0,142207.0,1118724.0,7051.0,11.18724
3152102,20220210,LIMA,LIMA,LA MOLINA,PCR,26.0,MASCULINO,2021-11-08,150114.0,17334234.0,1545976.0,9674755.0,14546.0,96.74755
136938,20220210,APURIMAC,ABANCAY,ABANCAY,PR,41.0,FEMENINO,2021-11-08,30101.0,20579563.0,36699.0,120116.0,8520.0,1.20116
2808907,20220210,PIURA,PIURA,CASTILLA,PCR,28.0,FEMENINO,2021-11-08,200104.0,13902570.0,152935.0,894847.0,7468.0,8.94847
2274077,20220210,HUANUCO,HUANUCO,HUANUCO,PCR,27.0,FEMENINO,2021-11-08,100101.0,8163443.0,50522.0,315799.0,6645.0,3.15799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494820,20220210,PUNO,LAMPA,LAMPA,AG,14.0,FEMENINO,2022-02-06,210701.0,24475734.0,63468.0,42549.0,5127.0,0.42549
1852138,20220210,AREQUIPA,AREQUIPA,AREQUIPA,PCR,38.0,MASCULINO,2022-02-06,40101.0,4719353.0,203178.0,1175765.0,13568.0,11.75765
2525806,20220210,ANCASH,HUARAZ,HUARAZ,PCR,51.0,MASCULINO,2022-02-06,20101.0,10897400.0,120596.0,185276.0,10214.0,1.85276
1869195,20220210,AMAZONAS,BAGUA,IMAZA,AG,36.0,MASCULINO,2022-02-06,10205.0,4692004.0,42543.0,84672.0,9968.0,0.84672


In [21]:
filtered_dates['PROVINCIA'] = filtered_dates['PROVINCIA'].replace('EN INVESTIGACIÃN', np.nan)
filtered_dates

<ipython-input-21-bdb0c20004fe>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dates['PROVINCIA'] = filtered_dates['PROVINCIA'].replace('EN INVESTIGACIÃN', np.nan)


,FECHA_CORTE,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX_x,EDAD,SEXO,FECHA_RESULTADO,UBIGEO,id_persona,METODODX_y,POBLACION,INDICE,POBLACION_CIENMIL
2449207,20220210,LA LIBERTAD,TRUJILLO,TRUJILLO,PCR,72.0,MASCULINO,2021-11-08,130101.0,10367661.0,142207.0,1118724.0,7051.0,11.18724
3152102,20220210,LIMA,LIMA,LA MOLINA,PCR,26.0,MASCULINO,2021-11-08,150114.0,17334234.0,1545976.0,9674755.0,14546.0,96.74755
136938,20220210,APURIMAC,ABANCAY,ABANCAY,PR,41.0,FEMENINO,2021-11-08,30101.0,20579563.0,36699.0,120116.0,8520.0,1.20116
2808907,20220210,PIURA,PIURA,CASTILLA,PCR,28.0,FEMENINO,2021-11-08,200104.0,13902570.0,152935.0,894847.0,7468.0,8.94847
2274077,20220210,HUANUCO,HUANUCO,HUANUCO,PCR,27.0,FEMENINO,2021-11-08,100101.0,8163443.0,50522.0,315799.0,6645.0,3.15799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494820,20220210,PUNO,LAMPA,LAMPA,AG,14.0,FEMENINO,2022-02-06,210701.0,24475734.0,63468.0,42549.0,5127.0,0.42549
1852138,20220210,AREQUIPA,AREQUIPA,AREQUIPA,PCR,38.0,MASCULINO,2022-02-06,40101.0,4719353.0,203178.0,1175765.0,13568.0,11.75765
2525806,20220210,ANCASH,HUARAZ,HUARAZ,PCR,51.0,MASCULINO,2022-02-06,20101.0,10897400.0,120596.0,185276.0,10214.0,1.85276
1869195,20220210,AMAZONAS,BAGUA,IMAZA,AG,36.0,MASCULINO,2022-02-06,10205.0,4692004.0,42543.0,84672.0,9968.0,0.84672


In [22]:
filtered_dates.isnull().sum()

FECHA_CORTE              0
DEPARTAMENTO           312
PROVINCIA            26262
DISTRITO             26262
METODODX_x               0
EDAD                     0
SEXO                     0
FECHA_RESULTADO          0
UBIGEO               26262
id_persona           16928
METODODX_y             313
POBLACION            27644
INDICE                 313
POBLACION_CIENMIL    27644
dtype: int64

In [23]:
weekly_sales = filtered_dates.groupby(["SEXO","DEPARTAMENTO","PROVINCIA", "POBLACION", "POBLACION_CIENMIL", "INDICE", pd.Grouper(key="FECHA_RESULTADO",freq="W-SUN")]).size()
weekly_sales = weekly_sales.unstack(0).fillna(0)
weekly_sales.sort_values(by=['FECHA_RESULTADO'], inplace=True, ascending=True)
weekly_sales.loc[:,'TOTAL'] = weekly_sales.sum(numeric_only=True, axis=1)

weekly_sales = weekly_sales.reset_index()
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,9968.0,2021-11-14,10.0,11.0,21.0
1,CUSCO,ANTA,63131.0,0.63131,8398.0,2021-11-14,0.0,1.0,1.0
2,CUSCO,CALCA,71582.0,0.71582,8398.0,2021-11-14,0.0,1.0,1.0
3,CUSCO,CANCHIS,106476.0,1.06476,8398.0,2021-11-14,4.0,2.0,6.0
4,CUSCO,CUSCO,511019.0,5.11019,8398.0,2021-11-14,34.0,45.0,79.0
...,...,...,...,...,...,...,...,...,...
1843,AYACUCHO,LA MAR,75277.0,0.75277,7046.0,2022-02-06,69.0,63.0,132.0
1844,LA LIBERTAD,BOLIVAR,15982.0,0.15982,7051.0,2022-02-06,22.0,16.0,38.0
1845,PUNO,MELGAR,69693.0,0.69693,5127.0,2022-02-06,133.0,134.0,267.0
1846,SAN MARTIN,MOYOBAMBA,133631.0,1.33631,6755.0,2022-02-06,214.0,170.0,384.0


In [24]:
weekly_sales.groupby("FECHA_RESULTADO").count()

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FEMENINO,MASCULINO,TOTAL
FECHA_RESULTADO,,,,,,,,
2021-11-14,117,117,117,117,117,117,117,117
2021-11-21,126,126,126,126,126,126,126,126
2021-11-28,113,113,113,113,113,113,113,113
2021-12-05,115,115,115,115,115,115,115,115
2021-12-12,112,112,112,112,112,112,112,112
2021-12-19,118,118,118,118,118,118,118,118
2021-12-26,114,114,114,114,114,114,114,114
2022-01-02,125,125,125,125,125,125,125,125
2022-01-09,151,151,151,151,151,151,151,151


In [25]:
cero_cases = weekly_sales['PROVINCIA']== "ATALAYA"
data_cero = weekly_sales[cero_cases]
data_cero

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
241,UCAYALI,ATALAYA,61049.0,0.61049,6590.0,2021-11-21,2.0,1.0,3.0
1109,UCAYALI,ATALAYA,61049.0,0.61049,6590.0,2022-01-16,1.0,0.0,1.0
1308,UCAYALI,ATALAYA,61049.0,0.61049,6590.0,2022-01-23,6.0,3.0,9.0
1522,UCAYALI,ATALAYA,61049.0,0.61049,6590.0,2022-01-30,4.0,2.0,6.0
1721,UCAYALI,ATALAYA,61049.0,0.61049,6590.0,2022-02-06,2.0,2.0,4.0


In [26]:
weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']] = weekly_sales[['FEMENINO','MASCULINO', 'TOTAL']].div(weekly_sales['POBLACION_CIENMIL'].values,axis=0)
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,9968.0,2021-11-14,11.810280,12.991308,24.801587
1,CUSCO,ANTA,63131.0,0.63131,8398.0,2021-11-14,0.000000,1.584008,1.584008
2,CUSCO,CALCA,71582.0,0.71582,8398.0,2021-11-14,0.000000,1.396999,1.396999
3,CUSCO,CANCHIS,106476.0,1.06476,8398.0,2021-11-14,3.756715,1.878358,5.635073
4,CUSCO,CUSCO,511019.0,5.11019,8398.0,2021-11-14,6.653373,8.805935,15.459308
...,...,...,...,...,...,...,...,...,...
1843,AYACUCHO,LA MAR,75277.0,0.75277,7046.0,2022-02-06,91.661464,83.690902,175.352365
1844,LA LIBERTAD,BOLIVAR,15982.0,0.15982,7051.0,2022-02-06,137.654862,100.112627,237.767488
1845,PUNO,MELGAR,69693.0,0.69693,5127.0,2022-02-06,190.836956,192.271821,383.108777
1846,SAN MARTIN,MOYOBAMBA,133631.0,1.33631,6755.0,2022-02-06,160.142482,127.215990,287.358472


In [27]:
weekly_sales.FEMENINO = weekly_sales.FEMENINO.round()
weekly_sales.MASCULINO = weekly_sales.MASCULINO.round()
weekly_sales.TOTAL = weekly_sales.FEMENINO + weekly_sales.MASCULINO
weekly_sales.FECHA_RESULTADO = weekly_sales.FECHA_RESULTADO.dt.strftime('%Y-%m-%d')
weekly_sales

SEXO,DEPARTAMENTO,PROVINCIA,POBLACION,POBLACION_CIENMIL,INDICE,FECHA_RESULTADO,FEMENINO,MASCULINO,TOTAL
0,AMAZONAS,BAGUA,84672.0,0.84672,9968.0,2021-11-14,12.0,13.0,25.0
1,CUSCO,ANTA,63131.0,0.63131,8398.0,2021-11-14,0.0,2.0,2.0
2,CUSCO,CALCA,71582.0,0.71582,8398.0,2021-11-14,0.0,1.0,1.0
3,CUSCO,CANCHIS,106476.0,1.06476,8398.0,2021-11-14,4.0,2.0,6.0
4,CUSCO,CUSCO,511019.0,5.11019,8398.0,2021-11-14,7.0,9.0,16.0
...,...,...,...,...,...,...,...,...,...
1843,AYACUCHO,LA MAR,75277.0,0.75277,7046.0,2022-02-06,92.0,84.0,176.0
1844,LA LIBERTAD,BOLIVAR,15982.0,0.15982,7051.0,2022-02-06,138.0,100.0,238.0
1845,PUNO,MELGAR,69693.0,0.69693,5127.0,2022-02-06,191.0,192.0,383.0
1846,SAN MARTIN,MOYOBAMBA,133631.0,1.33631,6755.0,2022-02-06,160.0,127.0,287.0


In [28]:
weekly_sales.to_csv('dataset_covid_total.csv' , index=False)

In [29]:
weekly_sales.to_json('dataset_covid_total.json', orient="table")